In [28]:
import pandas as pd
import numpy as np
import nltk
import sklearn
import string, re
import urllib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from nltk.corpus import stopwords

In [2]:
data = pd.read_csv('../Data/listings42018sum.csv')

In [3]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2318,"Casa Madrona - Urban Oasis, 1 block from the P...",2536,Megan,Central Area,Madrona,47.610819,-122.290816,Entire home/apt,296,4,15,2018-04-04,0.13,1,283
1,4291,Sunrise in Seattle Master Suite,35749,Jess & Joey,Other neighborhoods,Roosevelt,47.687801,-122.313427,Private room,82,2,54,2017-08-27,0.93,5,280
2,5682,"Cozy Studio, min. to downtown -WiFi",8993,Maddy,Delridge,South Delridge,47.523980,-122.359891,Entire home/apt,48,3,423,2018-03-29,4.30,1,157
3,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.654109,-122.337605,Entire home/apt,90,2,109,2017-12-07,1.02,3,324
4,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.550620,-122.320135,Private room,65,2,118,2018-02-08,1.26,5,357


In [4]:
data['availability_365'].value_counts

<bound method IndexOpsMixin.value_counts of 0       283
1       280
2       157
3       324
4       357
       ... 
7907     67
7908     58
7909     32
7910    365
7911     72
Name: availability_365, Length: 7912, dtype: int64>

In [5]:
data['reviews_per_month'] = data['reviews_per_month'].fillna(0)

In [6]:
ohe = OneHotEncoder(sparse=False)
neigh_group = ohe.fit_transform(data[['neighbourhood_group']])
neigh_group_cat = ohe.categories_
neigh = ohe.fit_transform(data[['neighbourhood']])
neigh_cat = ohe.categories_
room = ohe.fit_transform(data[['room_type']])
room_cat = ohe.categories_

In [7]:
def rename(name_of_columns,pre_addition):
    new_list = []
    for x in name_of_columns:
        for x in x:
            new_list.append(pre_addition+ '' + x)
    return new_list

In [8]:
new_neigh_group_cat = rename(neigh_group_cat,'neighbourhood_group: ')
new_neigh_cat = rename(neigh_cat,'neighbourhood: ')
new_room_cat = rename(room_cat,'room_type: ')

In [9]:
# Create categories for neighborhood_group, neighborhood and room_type
neigh_group_df = pd.DataFrame(data=neigh_group,columns=new_neigh_group_cat)
neigh_df = pd.DataFrame(data=neigh,columns=new_neigh_cat)
room_type_df = pd.DataFrame(data=room,columns = new_room_cat);

In [10]:
stopwords_list = stopwords.words('english') + list(string.punctuation)
vectorizer = TfidfVectorizer(strip_accents='unicode',stop_words=stopwords_list,min_df=10)

In [11]:
# get rid of na in name column
data.fillna({'name':''}, inplace=True)

In [12]:
tf_idf = vectorizer.fit_transform(data['name'])

In [13]:
nlp_name = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names())

In [14]:
clean_data = data.drop(columns = ['id','host_name','host_id','last_review','name','neighbourhood_group','neighbourhood','room_type'])

In [15]:
clean_data = pd.concat([clean_data,neigh_group_df,neigh_df,room_type_df,nlp_name],axis=1)

In [16]:
clean_data.describe()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group: Ballard,neighbourhood_group: Beacon Hill,...,westlake,wheel,whole,wifi,woodland,world,ws,yacht,yard,zen
count,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,...,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000
mean,47.626342,-122.333699,149.262260,2.667341,36.085819,1.895068,25.615521,128.397750,0.055865,0.035010,...,0.001059,0.000814,0.001036,0.001468,0.000814,0.000907,0.001463,0.001186,0.001098,0.001283
std,0.045824,0.031670,217.448831,8.468389,56.551004,3.771961,77.726732,123.121737,0.229674,0.183817,...,0.027439,0.021814,0.024899,0.028274,0.023077,0.025815,0.027314,0.029863,0.025437,0.028870
min,47.492871,-122.420918,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.606152,-122.353818,78.000000,1.000000,2.000000,0.230000,1.000000,12.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,47.621114,-122.331720,114.000000,2.000000,13.000000,1.030000,1.000000,89.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,47.661333,-122.312534,182.000000,2.000000,46.000000,2.870000,4.000000,246.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,47.737546,-122.234866,9999.000000,400.000000,690.000000,279.090000,366.000000,365.000000,1.000000,1.000000,...,0.860628,0.593837,0.784223,0.769957,0.777019,0.836446,0.613468,1.000000,0.718507,0.904485


In [17]:
X = clean_data.drop(columns = ['price'])
y = clean_data['price']

In [18]:
ss = StandardScaler()
X_ss = ss.fit_transform(X)

In [19]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X_ss,y)

In [20]:
from sklearn.linear_model import LinearRegression

In [21]:
lr = LinearRegression()
lr.fit(Xtrain,ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
lr.score(Xtrain,ytrain)

0.16412561195395403

In [51]:
rfr =RandomForestRegressor()

In [59]:
rfr.fit(Xtrain,ytrain)
rfr.score(Xtrain,ytrain)

0.8247546416785339

In [60]:
rfr_param_grid = {
    'n_estimators': [10, 100, 1001],
    'criterion': ['mse'],
    'max_depth': [None, 2, 6, 10],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [3,6]
}

In [61]:
rf_grid = GridSearchCV(rfr,rfr_param_grid,cv=3)

In [62]:
rf_grid.fit(Xtrain,ytrain)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['mse'], 'max_depth': 

In [63]:
rf_grid.best_params_

{'criterion': 'mse',
 'max_depth': None,
 'min_samples_leaf': 6,
 'min_samples_split': 5,
 'n_estimators': 100}

In [64]:
rf_grid.

0.03949287750139663

In [ ]:
sorted(list(zip(rfr.feature_importances_,clean_data.columns)),reverse=True);